<a href="https://colab.research.google.com/github/SonjaWangJQ/Keyboard-ML/blob/main/5_MFCC_data_amplifying.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### load dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

import math, random
import torch
import torchaudio
import librosa
from torchaudio import transforms
from IPython.display import Audio
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset, random_split
import csv






In [ ]:


class AudioUtil():
  # ----------------------------
  # Load an audio file. Return the signal as a tensor and the sample rate
  # ----------------------------

  @staticmethod
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    return (sig, sr)



    # ----------------------------
  # 数据放大*1.1 and *0.9
  # *1.1
  #
  @staticmethod
  def time_Amp(aud, Amp_size):
    sig,sr = aud
    Amp_sig = sig * Amp_size
    return (Amp_sig , sr)




  @staticmethod
  def compute_mfcc(aud, n_mfcc=32, window_size=0.01, step_size=0.0025, n_filters=32):

            sig, sr = aud
            n_fft = int(sr * window_size)
            hop_length = int(sr * step_size)

            mfccs = librosa.feature.mfcc(
                y=sig.numpy(),
                sr=sr,
                n_mfcc=n_mfcc,
                n_fft=n_fft,
                hop_length=hop_length,
                n_mels=n_filters
            )
            return mfccs.T
  @staticmethod
  def compute_stft(aud, n_fft=2048, hop_length=512, window='hann'):
    sig, sr = aud
    audio = sig.squeeze().numpy()

    # Compute STFT
    stft_result = librosa.core.stft(audio, n_fft=2048, hop_length=512, window='hann')

    # Get magnitude spectrum
    magnitude = np.abs(stft_result)

    # Get frequency axis
    freqs = librosa.core.fft_frequencies(sr=sr, n_fft=2048)

    return freqs, magnitude @staticmethod
  def compute_mfcc(aud,  window_size=0.01, step_size=0.0025, n_filters=32):

            sig, sr = aud
            n_fft = int(sr * window_size)
            hop_length = int(sr * step_size)

            mfccs = librosa.feature.mfcc(
                y=sig.numpy(),
                sr=sr,
                n_mfcc=n_mfcc,
                n_fft=n_fft,
                hop_length=hop_length,
                n_mels=n_filters
            )
            return mfccs.T


  @staticmethod
  def compute_fft(aud):

            # 计算 FFT
            sig, sr = aud
            audio = sig.squeeze().numpy()
            fft_result = np.fft.fft(audio)

            # 获取频率轴
            freqs = np.fft.fftfreq(len(audio), 1/sr)

            # 获取频率强度谱
            magnitude = np.abs(fft_result)

            return freqs, magnitude


In [ ]:

# ----------------------------
# SoundDS1 no ampf
# ----------------------------
class SoundDS(Dataset):

  def __init__(self, csv_filename):
    self.csv_filename = csv_filename
    self.sr = 44100
    self.channel = 1
    # self.shift_pct = -0.01   # +10ms
    self.audio_file, self.class_id = self.load_datapath()


  def load_datapath(self):
        audio_file, class_id = [], []
        with open(self.csv_filename) as f:
            reader = csv.reader(f)
            next(reader)  # Skip the header line
            for row in reader:
                i, l = row
                audio_file.append(i)
                class_id.append(int(l))  # Convert class ID to integer
        return audio_file, class_id

  def __len__(self):
    return len(self.audio_file)


  def get_class_counts(self):
        class_counts = {}
        for cls_id in self.class_id:
            if cls_id in class_counts:
                class_counts[cls_id] += 1
            else:
                class_counts[cls_id] = 1
        return class_counts


  def __getitem__(self, idx):

    # Absolute file path of the audio file - concatenate the audio directory with
    # the relative path，Get the Class ID

    # Get the Class ID
    audio_path = self.audio_file[idx]
    class_id = self.class_id[idx]


    aud = AudioUtil.open(audio_path)
    Amp_aud = AudioUtil.time_Amp(aud,self.amp_size)
    mfcc_aud = AudioUtil.compute_mfcc(Amp_aud)
    mfcc_data = np.array(mfcc_aud)
    # fft_audio = AudioUtil.compute_fft(shift_aud )
    # fft_data = np.array(fft_audio)
    # return mfcc_aud, class_id
    return mfcc_aud, class_id, audio_path # for mapping

In [ ]:

# ----------------------------
# SoundDS1 1.1 ampf
# ----------------------------
class SoundDS1(Dataset):

  def __init__(self, csv_filename):
    self.csv_filename = csv_filename
    self.sr = 44100
    self.channel = 1
    self.amp_size = 1.1  # 1.1
    self.audio_file, self.class_id = self.load_datapath()


  def load_datapath(self):
        audio_file, class_id = [], []
        with open(self.csv_filename) as f:
            reader = csv.reader(f)
            next(reader)  # Skip the header line
            for row in reader:
                i, l = row
                audio_file.append(i)
                class_id.append(int(l))  # Convert class ID to integer
        return audio_file, class_id

  def __len__(self):
    return len(self.audio_file)


  def get_class_counts(self):
        class_counts = {}
        for cls_id in self.class_id:
            if cls_id in class_counts:
                class_counts[cls_id] += 1
            else:
                class_counts[cls_id] = 1
        return class_counts


  def __getitem__(self, idx):

    # Absolute file path of the audio file - concatenate the audio directory with
    # the relative path，Get the Class ID

    # Get the Class ID
    audio_path = self.audio_file[idx]
    class_id = self.class_id[idx]


    aud = AudioUtil.open(audio_path)
    Amp_aud = AudioUtil.time_Amp(aud, self.amp_size)
    mfcc_aud = AudioUtil.compute_mfcc(Amp_aud)
    mfcc_data = np.array(mfcc_aud)
    # fft_audio = AudioUtil.compute_fft(shift_aud )
    # fft_data = np.array(fft_audio)
    # return mfcc_aud, class_id
    return mfcc_aud, class_id, audio_path # for mapping

In [ ]:

# ----------------------------
# SoundDS1 0.9 ampf
# ----------------------------
class SoundDS2(Dataset):

  def __init__(self, csv_filename):
    self.csv_filename = csv_filename
    self.sr = 44100
    self.channel = 1
    self.amp_size = 0.9 # 0.9
    self.audio_file, self.class_id = self.load_datapath()


  def load_datapath(self):
        audio_file, class_id = [], []
        with open(self.csv_filename) as f:
            reader = csv.reader(f)
            next(reader)  # Skip the header line
            for row in reader:
                i, l = row
                audio_file.append(i)
                class_id.append(int(l))  # Convert class ID to integer
        return audio_file, class_id

  def __len__(self):
    return len(self.audio_file)


  def get_class_counts(self):
        class_counts = {}
        for cls_id in self.class_id:
            if cls_id in class_counts:
                class_counts[cls_id] += 1
            else:
                class_counts[cls_id] = 1
        return class_counts


  def __getitem__(self, idx):

    # Absolute file path of the audio file - concatenate the audio directory with
    # the relative path，Get the Class ID

    # Get the Class ID
    audio_path = self.audio_file[idx]
    class_id = self.class_id[idx]


    aud = AudioUtil.open(audio_path)
    Amp_aud = AudioUtil.time_Amp(aud, self.amp_size)
    mfcc_aud = AudioUtil.compute_mfcc(Amp_aud)
    mfcc_data = np.array(mfcc_aud)
    # fft_audio = AudioUtil.compute_fft(shift_aud )
    # fft_data = np.array(fft_audio)
    # return mfcc_aud, class_id
    return mfcc_aud, class_id, audio_path # for mapping

In [ ]:

# ----------------------------
# * 1
# ----------------------------
class SoundDS(Dataset):

  def __init__(self, csv_filename):
    self.csv_filename = csv_filename
    self.sr = 44100
    self.channel = 1
    self.amp_size = 1
    self.audio_file, self.class_id = self.load_datapath()


  def load_datapath(self):
        audio_file, class_id = [], []
        with open(self.csv_filename) as f:
            reader = csv.reader(f)
            next(reader)  # Skip the header line
            for row in reader:
                i, l = row
                audio_file.append(i)
                class_id.append(int(l))  # Convert class ID to integer
        return audio_file, class_id

  def __len__(self):
    return len(self.audio_file)

  def __getitem__(self, idx):

    # Absolute file path of the audio file - concatenate the audio directory with
    # the relative path，Get the Class ID

    # Get the Class ID
    audio_path = self.audio_file[idx]
    class_id = self.class_id[idx]

    aud = AudioUtil.open(audio_path)
    # Amp_aud = AudioUtil.time_Amp(aud, self.amp_size)
    mfcc_aud = AudioUtil.compute_mfcc(aud)
    mfcc_data = np.array(mfcc_aud)
    # fft_audio = AudioUtil.compute_fft(shift_aud )
    # fft_data = np.array(fft_audio)

    return mfcc_aud, class_id, audio_path


In [ ]:
# shift  append into X,y
csv_filename = "/content/drive/MyDrive/collect_machinekeyboard/collect/piece2_441.csv"
sound_dataset_shift= SoundDS(csv_filename)
X = []
y = []
path = []


for idx in range(len(sound_dataset_shift)):
    mfcc_data, class_id,audio_path = sound_dataset_shift[idx]
    X.append(mfcc_data)
    y.append(class_id)
    path.append(audio_path)



In [ ]:
X = np.array(X)
y = np.array(y)
path = np.array(path)
# check length
assert X.shape[0] == len(y), "Shape mismatch between X and y"
assert X.shape[0] == len(path), "Shape mismatch between X and path"
print(X.shape)
num_samples, num_frames, num_coefficients, num_channels = X.shape
MFCC_data_2d = X.reshape(num_samples, num_frames * num_coefficients)


X_shuffled, y_shuffled, path_shuffled  = shuffle(MFCC_data_2d, y, path, random_state=42)
# X_shuffled, y_shuffled = shuffle(MFCC_data_2d, y, random_state=42)
X_train, X_test, y_train, y_test, path_train, path_test = train_test_split(X_shuffled, y_shuffled, path_shuffled, train_size=0.8, random_state=42,stratify = y_shuffled)

(1467, 41, 32, 1)


In [ ]:
print(y_train.shape,y_test.shape,path_test.shape)

(1173,) (294,) (294,)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # 标准化
X_test_scaled = scaler.transform(X_test) # keep it
print(X_train_scaled.shape, X_test_scaled.shape)

(1173, 1312) (294, 1312)


### append new data

In [ ]:
csv_filename = "/content/drive/MyDrive/collect_machinekeyboard/collect/piece2_441.csv"
sound_dataset_shift2 = SoundDS1(csv_filename)
X2 = []
y2 = []

for idx in range(len(sound_dataset_shift2)):
    mfcc_data2, class_id2, audio_path2 = sound_dataset_shift2[idx]  # 调用 __getitem__ 方法
    if audio_path2 in path_train:
        X2.append(mfcc_data2)
        y2.append(class_id2)

In [ ]:
# *0.9
csv_filename = "/content/drive/MyDrive/collect_machinekeyboard/collect/piece2_441.csv"
sound_dataset_shift3= SoundDS2(csv_filename)

for idx in range(len(sound_dataset_shift3)):
    mfcc_data3, class_id3, audio_path3 = sound_dataset_shift3[idx]  # 调用 __getitem__ 方法
    if audio_path3 in path_train:
        X2.append(mfcc_data3)
        y2.append(class_id3)


In [ ]:
X2 = np.array(X2)
y2 = np.array(y2)
# 检查 X 的行数与 y 的长度是否一致
assert X2.shape[0] == len(y2), "Shape mismatch between X and y"
print(X2.shape)
num_samples, num_frames, num_coefficients, num_channels = X2.shape

new_X2train = X2.reshape(num_samples, num_frames * num_coefficients)
print(new_X2train.shape)

(2346, 41, 32, 1)
(2346, 1312)


In [ ]:
X2_shuffled, y2_shuffled = shuffle( new_X2train , y2, random_state=42)
new_X2_train_scaled = scaler.fit_transform(X2_shuffled)
print(new_X2_train_scaled.shape, y2_shuffled)


(2346, 1312) [11  5 15 ... 34  6 17]


In [ ]:
# append X_train, y_train
X_train_extended = np.vstack([X_train_scaled, new_X2_train_scaled])
y_train_extended = np.concatenate([y_train, y2_shuffled])

In [ ]:
num_classes = len(np.unique(y_train_extended)) # 或者使用 len(np.unique(y_test))
print(y_train_extended)
print("Number of classes:", num_classes)

[20 12 32 ... 34  6 17]
Number of classes: 36


In [ ]:
print(X_train_extended.shape,y_train_extended.shape)
print(X_test_scaled.shape, y_test.shape)
# print(X_val_scaled.shape,X_test_scaled.shape, y_test.shape,y_valid.shape,)

(3519, 1312) (3519,)
(294, 1312) (294,)


## amf train

### svm

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

In [ ]:
#SVM model
# data shift
# X_train_extended.shape,y_train_extended.shape
# Best parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
from sklearn.metrics import top_k_accuracy_score

param_grid = {
    'C': [1],
    'kernel': ['linear'],
    # 'gamma': ['scale', 'auto', 0.1, 0.01, 0.001]
    'gamma':["scale"]
    }

#  SVM model
svm = SVC()

#  grid search object with 10-fold cross-validation
# grid_search = GridSearchCV(svm, param_grid, cv=10, n_jobs=-1)
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=10, n_jobs=-1) # 新修改
grid_search.fit(X_train_extended, y_train_extended)
# grid_search.fit(X_train_scaled, y_train)

#  best model from grid search
best_model = grid_search.best_estimator_

# Predict using the best model
y_pred = best_model.predict(X_test_scaled)
target_names = [f'Class {i}' for i in range(36)]
#   classification report
cr = classification_report(y_test, y_pred, target_names=target_names)
# TOP 5 ACCURACY
N_values = [1, 2, 3, 4, 5]
top_n_accuracies = [top_k_accuracy_score(y_test, best_model.decision_function(X_test_scaled), k=N) for N in N_values]

# Print results
print("parameters:", grid_search.best_params_)
print("Classification Report:")
print(cr)
for N, accuracy in zip(N_values, top_n_accuracies):
    print(f"Top-{N} Accuracy Score: {accuracy:.4f}")

parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.75      0.75      0.75         8
     Class 1       0.82      1.00      0.90         9
     Class 2       1.00      1.00      1.00         9
     Class 3       0.73      0.80      0.76        10
     Class 4       0.90      1.00      0.95         9
     Class 5       0.89      1.00      0.94         8
     Class 6       1.00      0.78      0.88         9
     Class 7       0.90      0.90      0.90        10
     Class 8       0.82      0.90      0.86        10
     Class 9       0.77      1.00      0.87        10
    Class 10       1.00      0.86      0.92         7
    Class 11       1.00      0.89      0.94         9
    Class 12       1.00      0.75      0.86         8
    Class 13       0.83      0.71      0.77         7
    Class 14       0.70      0.78      0.74         9
    Class 15       0.90      1.00      0.95         9

### MLR

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_digits
from sklearn.metrics import classification_report
from sklearn.utils import shuffle




multi_logreg = LogisticRegression(multi_class="multinomial", solver="lbfgs",max_iter=1000)

# parameters: {'C': 1, 'penalty': 'l2'}
param_grid = {
    'C': [ 1 ],
    'penalty': ['l2']
}

# 10 fold across
grid_search = GridSearchCV(multi_logreg, param_grid, cv=10, n_jobs=-1)

grid_search.fit(X_train_extended, y_train_extended) # *3
print("parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)



target_names = [f'Class {i}' for i in range(36)]

# report
cr = classification_report(y_test, y_pred, target_names=target_names)

# Top-N accuracy score
N_values = [1, 2, 3, 4, 5]
top_n_accuracies = [top_k_accuracy_score(y_test, best_model.predict_proba(X_test_scaled), k=N) for N in N_values]

# Print top-N accuracy scores
for N, accuracy in zip(N_values, top_n_accuracies):
    print(f"Top-{N} Accuracy Score: {accuracy:.4f}")

# Print classification report
print("Classification Report:")
print(cr)

parameters: {'C': 1, 'penalty': 'l2'}
Top-1 Accuracy Score: 0.8673
Top-2 Accuracy Score: 0.9252
Top-3 Accuracy Score: 0.9626
Top-4 Accuracy Score: 0.9728
Top-5 Accuracy Score: 0.9728
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.71      0.62      0.67         8
     Class 1       0.80      0.89      0.84         9
     Class 2       1.00      0.89      0.94         9
     Class 3       0.77      1.00      0.87        10
     Class 4       0.89      0.89      0.89         9
     Class 5       1.00      1.00      1.00         8
     Class 6       0.88      0.78      0.82         9
     Class 7       0.91      1.00      0.95        10
     Class 8       0.82      0.90      0.86        10
     Class 9       0.91      1.00      0.95        10
    Class 10       1.00      0.71      0.83         7
    Class 11       0.80      0.89      0.84         9
    Class 12       0.88      0.88      0.88         8
    Class 13       1.00      0.86    

### RF

In [ ]:
import pydot
import scipy.stats as stats
from pprint import pprint
from sklearn import metrics
from openpyxl import load_workbook
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import classification_report

# Assuming you have loaded X_train_scaled, y_train, X_test_scaled, and y_test

# Define the parameter grid for grid search
param_grid = {
    'n_estimators': [100],
    'max_depth': [None],
    'min_samples_split': [10],
    'min_samples_leaf': [4],
    'bootstrap': [False]
}
# {'bootstrap': False, 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
# Create and fit the Random Forest model using GridSearchCV
random_forest = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(random_forest, param_grid, cv=10, n_jobs=-1)
grid_search.fit(X_train_extended, y_train_extended)

best_random_forest = grid_search.best_estimator_

# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
y_test_pred_probs = cross_val_predict(best_random_forest, X_test_scaled, y_test, cv=kf, method='predict_proba')

# ... (Rest of your code for calculating top-N accuracy scores and generating the classification report)
N_values = [1, 2, 3, 4, 5]
top_n_accuracies = []
for N in N_values:
    top_n_correct = 0
    for true_label, pred_probs in zip(y_test, y_test_pred_probs):
        top_n_indices = np.argsort(pred_probs)[::-1][:N]  # Indices of top-N classes
        if true_label in top_n_indices:
            top_n_correct += 1
    top_n_accuracy = top_n_correct / len(y_test)
    top_n_accuracies.append(top_n_accuracy)

# Print the best hyperparameters found by GridSearchCV
print("Best Hyperparameters:")
print(grid_search.best_params_)

# Print top-N accuracy scores
for N, accuracy in zip(N_values, top_n_accuracies):
    print(f"Top-{N} Accuracy Score: {accuracy:.4f}")

# Generate classification report
target_names = [f'Class {i}' for i in range(36)]
y_test_pred = np.argmax(y_test_pred_probs, axis=1)
cr = classification_report(y_test, y_test_pred, target_names=target_names)
# Print classification report
print("Classification Report:")
print(cr)



Best Hyperparameters:
{'bootstrap': False, 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
Top-1 Accuracy Score: 0.2789
Top-2 Accuracy Score: 0.3946
Top-3 Accuracy Score: 0.4558
Top-4 Accuracy Score: 0.5000
Top-5 Accuracy Score: 0.5544
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.50      0.12      0.20         8
     Class 1       0.31      0.44      0.36         9
     Class 2       0.31      0.44      0.36         9
     Class 3       0.09      0.10      0.10        10
     Class 4       0.22      0.22      0.22         9
     Class 5       0.75      0.38      0.50         8
     Class 6       0.33      0.33      0.33         9
     Class 7       0.33      0.50      0.40        10
     Class 8       0.60      0.30      0.40        10
     Class 9       0.33      0.60      0.43        10
    Class 10       0.00      0.00      0.00         7
    Class 11       0.46      0.67      0.55         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
